In [0]:
from IPython.display import clear_output

In [0]:
!pip3 install pycodestyle flake8 pycodestyle_magic
!pip3 install pymorphy2[fast]
!pip3 install keras_bert
!pip3 install smart_open
!pip3 install h5py
clear_output()

In [0]:
%load_ext autoreload
%load_ext pycodestyle_magic

import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
from warnings import filterwarnings
from scipy.sparse import csr_matrix
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
from gensim.models.word2vec import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

In [0]:
tf.reset_default_graph()

In [0]:
filterwarnings("ignore")

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## word2vec + fasttext

загрузка модели

In [0]:
# если модель без тэгов
model_file = "./fasttext/model.model"

model = Word2Vec.load(model_file)

In [0]:
# если модель с POS-тэггингом
model_file = "./fasttext/model.model"

model = KeyedVectors.load(model_file)

проверка наличия слова в словаре

In [0]:
lemma = "черепаха"
lemma in model.vocab

True

получение вектора слова

In [0]:
model.wv[lemma]

In [0]:
model[lemma].shape

(300,)

получение вектора документа

In [0]:
import numpy as np

# сделали препроцессинг, получили леммы 
lemmas = ['старинный_ADJ', 'замок_NOUN']

# создаем маски для векторов 
lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
vec = np.zeros((model.vector_size,))

# если слово есть в модели, берем его вектор
for idx, lemma in enumerate(lemmas):
    if lemma in model.wv:
        lemmas_vectors[idx] = model.wv[lemma]
        
# проверка на случай, если на вход пришел пустой массив
if lemmas_vectors.shape[0] is not 0:
    vec = np.mean(lemmas_vectors, axis=0)


## Задание

Реализуйте поиск по [Quora question pairs](https://www.kaggle.com/loopdigga/quora-question-pairs-russian) на нескольких векторных моделях

    1. fasttext, модель ruscorpora_none_fasttextskipgram_300_2_2019
    2. elmo, модель ruwikiruscorpora_lemmas_elmo_1024_2019
    3. bert*, RuBERT - необязательно
   
Первые две обученные модели можно скачать на сайте [rusvectores](https://rusvectores.org/en/models/).

BERT делать необязательно, но если сделаете, 6 за курс у вас автоматом. Модель можно [найти тут](http://docs.deeppavlov.ai/en/master/features/models/bert.html).

In [0]:
!wget https://www.dropbox.com/s/cfjv7galp6ajyr0/quora_question_pairs_rus.csv?dl=0 -O quora_question_pairs_rus.csv
clear_output()

In [0]:
df = pd.read_csv("quora_question_pairs_rus.csv")
df = df.dropna()
df.head()

,Unnamed: 0,question1,question2,is_duplicate
0,0,Какова история кохинор кох-и-ноор-бриллиант,"что произойдет, если правительство Индии украд...",0
1,1,как я могу увеличить скорость моего интернет-с...,как повысить скорость интернета путем взлома ч...,0
2,2,"почему я мысленно очень одинок, как я могу это...","найти остаток, когда математика 23 ^ 24 матема...",0
3,3,которые растворяют в воде быстро сахарную соль...,какая рыба выживет в соленой воде,0
4,4,астрология: я - луна-колпачок из козерога и кр...,Я тройная луна-козерог и восхождение в козерог...,1


In [0]:
queries = list(set(df["question1"]))
query_idx = {queries[i]: i for i in range(len(queries))}
docs = list(set(df["question2"]))
doc_idx = {docs[i]: i for i in range(len(docs))}

In [0]:
with open("docs.pkl", "wb") as file:
    pickle.dump(docs, file)

In [0]:
file = drive.CreateFile({"title": "docs.pkl"})
file.SetContentFile("docs.pkl")
file.Upload()

In [0]:
df_dup = df[df["is_duplicate"] == 1]
row_ind = df_dup["question1"].apply(lambda x: query_idx[x])
col_ind = df_dup["question2"].apply(lambda x: doc_idx[x])
dup_matrix = csr_matrix((np.ones(df_dup.shape[0]), (row_ind, col_ind)),
                        shape=(len(query_idx), len(doc_idx)))

In [0]:
def accuracy(search_engine, queries, dup_matrix, test_size=10000):
    true_results = 0
    all_results = 0
    test = np.random.choice(queries, size=test_size)
    for query in test:
        if dup_matrix[query_idx[query], ].sum():
            all_results += 1
            results = search_engine.search(query)
            for result, score in results:
                if dup_matrix[query_idx[query], doc_idx[result]]:
                    true_results += 1
                    break
    return true_results / all_results

In [0]:
m = MorphAnalyzer()


def lemmatize(text):
    return [m.parse(word)[0].normal_form 
            for word in simple_word_tokenize(text)]

### __Задача 1__:    
Сравните время индексации корпуса для каждой модели 

### __Задача 2__:    
Выведите качество поиска для каждой модели +  BM25 для сравнения

Качество оцениваем так же, как в прошлом задании:
    - если в топ-5 результатов выдачи попал хоть один релевантный документ, выдача точная
    - если в топ-5 нет ни одного релеватного документа, выдача получает 0
   

##Fasttext

In [0]:
!mkdir fasttext
!wget http://vectors.nlpl.eu/repository/11/181.zip -O fasttext.zip
!unzip fasttext.zip -d fasttext
!rm fasttext.zip
clear_output()

In [0]:
class SearchFasttext():
    def __init__(self, data, model_file):
        self.texts = np.array(data)
        self.model = KeyedVectors.load(model_file)
        self.vec = np.zeros((len(self.texts), self.model.vector_size))
        for i in range(len(self.texts)):
            lemmas = lemmatize(self.texts[i])
            lemmas_vectors = np.zeros((len(lemmas), self.model.vector_size))
            for idx, lemma in enumerate(lemmas):
                if lemma in self.model.vocab:
                    lemmas_vectors[idx] = self.model[lemma]
            if lemmas_vectors.shape[0] is not 0:
                self.vec[i] = np.mean(lemmas_vectors, axis=0)            

    def search(self, query, n=5):
        lemmas = lemmatize(query)
        query_vec = np.zeros((self.model.vector_size, ))
        lemmas_vectors = np.zeros((len(lemmas), self.model.vector_size))
        for idx, lemma in enumerate(lemmas):
            if lemma in self.model.vocab:
                lemmas_vectors[idx] = self.model[lemma]
        if lemmas_vectors.shape[0] is not 0:
            query_vec = np.mean(lemmas_vectors, axis=0)
        query_vec = np.transpose(query_vec)
        result = np.matmul(self.vec, query_vec)
        indices = np.argsort(result)[::-1].tolist()[:n]
        return list(zip(self.texts[indices], result[indices]))

In [0]:
%%time
FasttextSearchEngine = SearchFasttext(docs, "./fasttext/model.model")

CPU times: user 1min 52s, sys: 3.48 s, total: 1min 56s
Wall time: 1min 56s


In [0]:
for result in FasttextSearchEngine.search("рождественские каникулы", n=10):
    print(result)

('вечеринки', 7.227940226714265)
('как празднуется Рождество', 6.348252635695177)
('сколько учеников ежегодно посещают весенние каникулы', 6.208211912807602)
('почему атеисты празднуют Рождество', 5.929559275163776)
('Законопроекты', 5.816046160192412)
('программа стипендий выпускников университетов', 5.815099947238545)
('как мусульмане празднуют Рождество', 5.813684691519306)
('какие праздники празднуют атеисты', 5.528922709816639)
('каков ваш обзор каникул', 5.251107693457673)
('почему вы празднуете рождество', 5.220994990144241)


In [0]:
accuracy(FasttextSearchEngine, queries, dup_matrix)

0.08765473402475744

##ELMO

In [0]:
!mkdir elmo
!wget http://vectors.nlpl.eu/repository/11/196.zip -O elmo.zip
!wget https://github.com/ltgoslo/simple_elmo/archive/master.zip
!unzip elmo.zip -d elmo
!unzip master.zip
!mv ./simple_elmo-master/* .
!gzip ./elmo/vocab.txt
!rm *.zip get_elmo_vectors.py LICENSE README.md requirements.txt test.txt vocabulary.py ./simple_elmo-master/.gitignore
!rmdir simple_elmo-master
clear_output()

In [0]:
from elmo_helpers import load_elmo_embeddings, get_elmo_vectors
from bilm import Batcher, BidirectionalLanguageModel, weight_layers

In [0]:
batcher, ids, elmo_input = load_elmo_embeddings("./elmo")
clear_output()

In [0]:
class SearchELMO():
    def __init__(self, data):
        self.texts = np.array(data)
        lemm = [lemmatize(text) for text in self.texts]
        self.vec = np.zeros((0, 1024))
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for i in range(0, len(self.texts), 75):
                self.vec = np.vstack((self.vec,
                                      np.mean(get_elmo_vectors(sess,
                                                               lemm[i:i + 75],
                                                               batcher, ids,
                                                               elmo_input),
                                              axis=1)))

    def search(self, query, n=5):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            query_vec = np.transpose(np.mean(get_elmo_vectors(sess,
                                                              [lemmatize(query)],
                                                              batcher, ids,
                                                              elmo_input),
                                             axis=1)).flatten()
            result = np.matmul(self.vec, query_vec)
            indices = np.argsort(result)[::-1].tolist()[:n]
        return list(zip(self.texts[indices], result[indices]))

In [0]:
%%time
ELMOSearchEngine = SearchELMO(docs)
clear_output()

CPU times: user 34min 19s, sys: 4min 10s, total: 38min 29s
Wall time: 53min 9s


In [0]:
file = drive.CreateFile({"title": "vec.pkl"})
file.SetContentFile("vec.pkl")
file.Upload()

In [0]:
for result in ELMOSearchEngine.search("рождественские каникулы", n=10):
    print(result)

Sentences in this batch: 1


('Я хочу присоединиться к некоторым хорошим курсам этического хакера в Дели для своих летних каникул, что является хорошим учебным институтом для моих праздников, поскольку я все еще новичок', 49.54284461506208)
('что я должен делать один час в день в школе, чтобы оптимизировать свой школьный опыт, у меня есть 8-часовой школьный день, и мой единственный перерыв на обед, что я могу сделать', 48.009692874846266)
('Таиланд медовый месяц в январе: какие лучшие скрытые драгоценные камни в бангкоке и пхукете у нас есть две недели, и мне нравится выйти из проторенного пути', 48.00078541187626)
('я работаю по понедельникам и понедельникам, возглавляю два руководящих подразделения на танцевальных вечеринках на утреннем роликовом клинке в пятницу, делаю стрельбу из лука в субботу и делаю черлидинг в воскресенье, сколько хобби слишком много', 47.63022367818892)
('почему мы всегда ест индейку в день благодарения, почему бы нам не съесть еще одну вкусную еду, чтобы отпраздновать благодарение', 46.1

In [0]:
elmo_accuracy = accuracy(ELMOSearchEngine, queries, dup_matrix)
clear_output()
elmo_accuracy

0.004369747899159664


In [0]:
with open("vec.pkl", "wb") as file:
    pickle.dump(ELMOSearchEngine.vec, file)

##RuBERT

In [0]:
from keras.models import Model
from keras_bert.layers import MaskedGlobalMaxPool1D
from keras_bert import load_trained_model_from_checkpoint
from keras_bert import Tokenizer, load_vocabulary, get_checkpoint_paths

In [0]:
!mkdir rubert
!wget http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v2.tar.gz -O rubert.tar
!tar -xf rubert.tar
!mv ./rubert_cased_L-12_H-768_A-12_v2/* rubert
!rm rubert.tar
!rmdir rubert_cased_L-12_H-768_A-12_v2
clear_output()

In [0]:
class SearchBERT():
    def __init__(self, data, model_path):
        self.texts = np.array(data)
        self.vec = np.zeros((len(self.texts), 768))
        paths = get_checkpoint_paths(model_path)
        inputs = load_trained_model_from_checkpoint(
            config_file=paths.config,
            checkpoint_file=paths.checkpoint, seq_len=50)
        outputs = MaskedGlobalMaxPool1D(name='Pooling')(inputs.output)
        self.model = Model(inputs=inputs.inputs, outputs=outputs)
        self.vocab = load_vocabulary(paths.vocab)
        self.tokenizer = Tokenizer(self.vocab)
        for i in range(len(self.texts)):
            tokens = self.tokenizer.tokenize(" ".join(lemmatize(
                self.texts[i])))[:50]
            indices = [self.vocab[token] for token in tokens] + \
                [0 for j in range(50 - len(tokens))]
            segments = [0 for j in range(50)]
            self.vec[i] = self.model.predict([np.array([indices]),
                                              np.array([segments])])[0]

    def search(self, query, n=5):
        tokens = self.tokenizer.tokenize(" ".join(lemmatize(query)))[:50]
        indices = [self.vocab[token] for token in tokens] + \
            [0 for i in range(50 - len(tokens))]
        segments = [0 for i in range(50)]
        query_vec = self.model.predict([np.array([indices]),
                                        np.array([segments])])[0]
        result = np.matmul(self.vec, query_vec)
        idxs = np.argsort(result)[::-1].tolist()[:n]
        return list(zip(self.texts[idxs], result[idxs]))

In [0]:
%%time
BERTSearchEngine = SearchBERT(docs, "./rubert")
clear_output()

CPU times: user 1h 33min 8s, sys: 8min 13s, total: 1h 41min 21s
Wall time: 1h 51min 59s


In [0]:
for result in BERTSearchEngine.search("рождественские каникулы", n=10):
    print(result)

('стоит потратить деньги на то, чтобы купить что-то в Интернете в течение большого миллиарда дней, а также другие продажи, предлагаемые flipkart amazon snapdeal и т. д.', 510.45399032816727)
('в чем разница между реальными и поддельными часами премиальных брендов, такими как omega rolex tag heuer и т. д. с точки зрения функциональности долговечности и т. д.', 507.5804296195971)
('когда вы переписываете один сайт на другой, Google использует старый сайт seo или отбрасывает его и использует только новые метаданные seo h1 h2 на сайте и т. д.', 507.47724148129316)
('вы подписались на приложение для веб-сайта, в котором вы получаете все свои каналы социальных сетей, например facebook twitter instagram google + quora и т. д.', 507.30373699738817)
('где я могу найти подробное руководство по передовым практикам в качестве фрилансера, когда дело доходит до таких сайтов, как elance guru freelancer odesk и т. д.', 504.7061882486736)
('как интернет-магазины, такие как flipkart и snapdeal, могут пр

In [0]:
bert_accuracy = accuracy(BERTSearchEngine, queries, dup_matrix)
clear_output()
bert_accuracy

0.0

Быстрее (≈ 2 минуты) и лучше (accuracy ≈ 0.09) всего работает модель FastText. Модель ELMO обучается за ≈ 1 час и даёт accuracy ≈ 0.01, а модель RuBERT обучается за ≈ 2 часа и даёт accuracy ≈ 0.00.